In [1]:
import numpy as np
import pandas as pd
import glob
import requests
import shutil
from bs4 import BeautifulSoup
import itertools
import json
import urllib.request

In [3]:
# doctolib pas adapte https://www.doctolib.fr/medecin-generaliste/paris
# ameli mieux mais pas pratique (post) http://annuairesante.ameli.fr/
# professionnels-de-sante/recherche-2/liste-resultats-page-1-par_page
# -20-tri-distance_asc-221df78764058bde19cfc8e8d7517b29.html
URL_ROOT = "http://annuairesante.ameli.fr"

def getSoupFromURL(url, method='get', data={}, parser='html.parser'):
    if method == 'get':
        res = requests.get(url)
    elif method == 'post':
        res = requests.post(url, data=data)
    else:
        return None

    if res.status_code == 200:
        soup = BeautifulSoup(res.text, parser)
        return soup
    else:
        return None

def getSpecialtyPage():
    output = None
    url = URL_ROOT + "/trouver-un-professionnel-de-sante"
    soup = getSoupFromURL(url)
    if(soup):
        tmp_tag = soup.find(class_="seo-liste")
        rows = tmp_tag.find_all("a")
        output = dict([(row.string, URL_ROOT + row.get('href')) 
                       for row in rows])
    return output

def getSpecialtyDptPage(url):
    output = None
    soup = getSoupFromURL(url)
    if(soup):
        rows = soup.find_all(class_="seo-departement")
        output = dict([(row.find("a").string, URL_ROOT + row.find("a").get("href")) 
                      for row in rows])
    return output

def getSpecialtyTownPage(url):
    output = None
    soup = getSoupFromURL(url)
    if(soup):
        rows = soup.find(class_="seo-liste").find_all("a")
        output = dict([(row.string, URL_ROOT + row.get("href")) 
                      for row in rows])
    return output

def scrapDoctors(url):
    output = None
    soup = getSoupFromURL(url, parser='lxml')
    if(soup):
        print(soup)
        print("")
        rows = soup.find(class_="liste-criteres")
        #.find_all("a")
        # rows = soup.find(class_="liste-professionnel").find_all(class_="item-professionnel-inner")
        print(rows)
        #output = dict([(row.find("a").get('href'), row.find(class_="item right type_honoraires").string)
        #              for row in rows])
    return output

d_spties = getSpecialtyPage()

d_dpt = getSpecialtyDptPage(d_spties['Cancérologues'])
# print(d_dpt)

d_ville = getSpecialtyTownPage(d_dpt['Dans le département 06 (Alpes Maritimes)'])
print(d_ville)

d_pro = scrapDoctors(d_ville['Cagnes Sur Mer (06)'])
# pb to get conventionnement info: javascript ... 
print(d_pro)


{'St Laurent Du Var (06700)': 'http://annuairesante.ameli.fr/trouver-un-professionnel-de-sante/cancerologues/06-alpes-maritimes-st-laurent-du-var', 'St Laurent Du Var Cede (06721)': 'http://annuairesante.ameli.fr/trouver-un-professionnel-de-sante/cancerologues/06-alpes-maritimes-st-laurent-du-var-cede', 'Cagnes Sur Mer (06)': 'http://annuairesante.ameli.fr/trouver-un-professionnel-de-sante/cancerologues/06-alpes-maritimes-cagnes-sur-mer', 'Nice (06)': 'http://annuairesante.ameli.fr/trouver-un-professionnel-de-sante/cancerologues/06-alpes-maritimes-nice', 'Mougins (06)': 'http://annuairesante.ameli.fr/trouver-un-professionnel-de-sante/cancerologues/06-alpes-maritimes-mougins', 'Antibes (06)': 'http://annuairesante.ameli.fr/trouver-un-professionnel-de-sante/cancerologues/06-alpes-maritimes-antibes'}
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xml:lang="fr" xmlns="http://www.w3.org/1999/xhtml">
<head>
<tit

In [ ]:



# effectif / specialisation, liberal O/N
df_doctors_tab1 = pd.read_csv("rpps/rpps-medecins-tab1_46204756137818.csv", 
                              sep=',', header=3, encoding='latin_1')
# non specialite, inscription, sexe, age
#df_doctors_tab2 = pd.read_csv("rpps/rpps-medecins-tab2_46205062232751.csv", 
#                              sep=',', header=3, encoding='latin_1')
# effectif / specialite / region
df_doctors_tab3 = pd.read_csv("rpps/rpps-medecins-tab3_46205308951370.csv", 
                              sep=',', header=3, encoding='latin_1')
#non
#df_doctors_tab4 = pd.read_csv("rpps/rpps-medecins-tab4_46205481295665.csv", 
#                              sep=',', header=3, encoding='latin_1')
#df_doctors_tab5 = pd.read_csv("rpps/rpps-medecins-tab5_46205748484002.csv", 
#                              sep=',', header=7, encoding='latin_1')
#non
#df_doctors_tab6 = pd.read_csv("rpps/rpps-medecins-tab6_46205934421334.csv", 
#                              sep=',', header=3, encoding='latin_1')
#densite medecins par region / dpt
df_doctors_tab7 = pd.read_csv("rpps/rpps-medecins-tab7_46206108640525.csv", 
                              sep=',', header=3, index_col='SPECIALITE', encoding='latin_1')
# df_doctors_tab8 = pd.read_csv("rpps/rpps-medecins-tab1_46204756137818.csv", sep=',', header=3, encoding='latin_1')
#df_doctors_tab9 = pd.read_csv("rpps/rpps-medecins-tab9_46206490361358.csv", 
#                              sep=',', header=3, encoding='latin_1')
#df_doctors_tab10 = pd.read_csv("rpps/rpps-medecins-tab10_46206652704647.csv", 
#                               sep=',', header=3, encoding='latin_1')
#df_doctors_tab11 = pd.read_csv("rpps/rpps-medecins-tab11_46206836766844.csv", 
#                               sep=',', header=3, encoding='latin_1')
# df_N201707 = pd.read_csv("N201707.csv", sep=';', header=1, encoding='latin_1')

N201707_type = {'l_serie': str,
'SERIE': str,                
'prs_nat': str,
'rem_date': str,
'l_prs_nat': str,
'sns_date': str,
'asu_nat': str,
'l_asu_nat': str,
'cpl_cod': str,
'l_cpl_cod': str,
'REM_TAU': str,
'pre_spe': str,
'l_pre_spe': str,
'pre_spe1': str,
'l_pre_spe1': str,
'pre_stj1': str,
'l_pre_stj1': str,
'exe_spe': str,
'l_exe_spe': str,
'exe_spe1': str,
'l_exe_spe1': str,
'exe_stj1': str,
'l_exe_stj1': str,
'rem_mon': str,
'rec_mon': str,
'dep_mon': str,
'act_dnb': str,
'act_coe': str}


df_N201707 = pd.read_csv("N201707.csv", sep=';', header=0, dtype = N201707_type, encoding='latin_1', decimal=",")

In [ ]:
df_N201707

In [ ]:
# df_doctors_tab7.drop(["ZONE INSCRIPT", "FRANCE ENTIERE"], inplace=True)
df_densite_dpt = df_doctors_tab7[df_doctors_tab7.index.str.contains("[0-9]")]

In [ ]:
df_densite_dpt

In [ ]:
s = pd.Series(['a1', 'b2', 'c3'])
s.str.extract('([ab])(\d)')
#generalise le .split
s2 = s.sort_values(ascending=False)
print(s2)

In [ ]:
#plotly d3js highcharts